# Notebook to import the csv file downloaded from wetherlink.com to mysql database. Data is collected using Vantage Pro


In [11]:
import mysql.connector as msql
from mysql.connector import Error

import pandas as pd
import datetime


Assume that mysql server is already running and create a new database. Password for DB needs to be entered.

In [12]:
try:
    conn = msql.connect(user='root', password='omitted',
                              host='localhost', 
                              auth_plugin='mysql_native_password')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE CapWeather")
        print("CapWeather database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

Error while connecting to MySQL 1007 (HY000): Can't create database 'CapWeather'; database exists


Read the downloaded data into a dataframe.

In [13]:
#weatherData = pd.read_csv('Capilano_University_Weather_Centre_1-1-20_12-00_AM_1_Year_1611609346_v2.csv',index_col=False,encoding = "ISO-8859-1", skiprows = 5)

weatherData = pd.read_csv('Capilano_University_Weather_Centre_1-1-20_12-00_AM_1_Year_1611714841_v2.csv',index_col=False,encoding = "ISO-8859-1", skiprows = 5)

weatherData.head()

,Date & Time,Barometer - mb,Temp - °C,High Temp - °C,Low Temp - °C,Hum - %,Dew Point - °C,Wet Bulb - °C,Wind Speed - km/h,Wind Direction,...,High Wind Speed - km/h,High Wind Direction,Wind Chill - °C,Heat Index - °C,THW Index - °C,Rain - mm,Rain Rate - mm/h,ET - mm,Heating Degree Days,Cooling Degree Days
0,1/1/20 12:00 AM,1002.6,6,6,6,100,6,6,0,NaN,...,0,NaN,6,6,6,0.0,0.0,0.0,0.252,0.000
1,1/1/20 12:30 AM,1002.5,7,7,6,100,7,7,0,NaN,...,0,NaN,7,7,7,0.0,0.0,0.0,0.233,0.000
2,1/1/20 1:00 AM,1002.6,8,8,7,100,8,8,0,NaN,...,0,NaN,8,8,8,0.0,0.0,0.0,0.222,0.000
3,1/1/20 1:30 AM,1002.4,8,8,8,100,8,8,0,NaN,...,6,E,8,8,8,0.0,0.0,0.0,0.218,0.000
4,1/1/20 2:00 AM,1002.7,8,8,8,100,8,8,2,SW,...,19,SSW,8,9,9,0.2,0.0,0.0,0.206,0.000


The dtype of the 'Date Time' column is object. It would make sense to change it to datetime, but the instruction below changes is to datetime64, which causes trouble when we want to write it to the database. I chose to leave the column as is and change the object to a datatime object before writing it to the database.

In [14]:
#change first column to date/time

#weatherData['Date Time'] = pd.to_datetime(weatherData['Date Time'])


The Wind direction rows contain NaN which messes up mysql import, so they are changed to string type.

In [15]:
weatherData['Wind Direction'] = weatherData['Wind Direction'].astype('|S5').str.decode('utf-8') 
weatherData['High Wind Direction'] = weatherData['High Wind Direction'].astype('|S5').str.decode('utf-8') 

It seems that weatherlink records missing data with --. Put the rows with missing data into a separeate df. Will try do add each row to the database separately.

In [16]:
#get bad rows
weatherDataMissing = weatherData[(weatherData == '--').any(axis=1)].copy()
weatherDataMissing

,Date & Time,Barometer - mb,Temp - °C,High Temp - °C,Low Temp - °C,Hum - %,Dew Point - °C,Wet Bulb - °C,Wind Speed - km/h,Wind Direction,...,High Wind Speed - km/h,High Wind Direction,Wind Chill - °C,Heat Index - °C,THW Index - °C,Rain - mm,Rain Rate - mm/h,ET - mm,Heating Degree Days,Cooling Degree Days
1873,11/3/20 7:40 AM,--,--,--,--,--,--,--,--,--,...,0,nan,--,--,--,0.0,0.0,0.0,--,--


Create a df with complete measurements. You can add to the list if there is a trouble value other than '--'. 
copy is not necessary, but it avoids view/copy warning issue later.

In [17]:

weatherDataComplete = weatherData[~weatherData.isin(['--']).any(1)].copy()
weatherDataComplete.head()

,Date & Time,Barometer - mb,Temp - °C,High Temp - °C,Low Temp - °C,Hum - %,Dew Point - °C,Wet Bulb - °C,Wind Speed - km/h,Wind Direction,...,High Wind Speed - km/h,High Wind Direction,Wind Chill - °C,Heat Index - °C,THW Index - °C,Rain - mm,Rain Rate - mm/h,ET - mm,Heating Degree Days,Cooling Degree Days
0,1/1/20 12:00 AM,1002.6,6,6,6,100,6,6,0,nan,...,0,nan,6,6,6,0.0,0.0,0.0,0.252,0.000
1,1/1/20 12:30 AM,1002.5,7,7,6,100,7,7,0,nan,...,0,nan,7,7,7,0.0,0.0,0.0,0.233,0.000
2,1/1/20 1:00 AM,1002.6,8,8,7,100,8,8,0,nan,...,0,nan,8,8,8,0.0,0.0,0.0,0.222,0.000
3,1/1/20 1:30 AM,1002.4,8,8,8,100,8,8,0,nan,...,6,E,8,8,8,0.0,0.0,0.0,0.218,0.000
4,1/1/20 2:00 AM,1002.7,8,8,8,100,8,8,2,SW,...,19,SSW,8,9,9,0.2,0.0,0.0,0.206,0.000


The columns were not imported from csv as numeric because of the -- notation for missing values. Since the rows that contain -- are now removed, we can change the rows that shoud be numeric to numeric type.



In [18]:



cols=[i for i in weatherData.columns if i not in ["Date Time", "Wind Direction", "High Wind Direction"]]
for col in cols:
    #aaa = pd.to_numeric(weatherDataComplete[col], errors = 'ignore').copy()
    weatherDataComplete[col] = pd.to_numeric(weatherDataComplete[col], errors = 'ignore')
   



weatherDataComplete.dtypes

Date & Time                object
Barometer - mb            float64
Temp - °C                   int64
High Temp - °C              int64
Low Temp - °C               int64
Hum - %                     int64
Dew Point - °C              int64
Wet Bulb - °C               int64
Wind Speed - km/h           int64
Wind Direction             object
Wind Run - km             float64
High Wind Speed - km/h      int64
High Wind Direction        object
Wind Chill - °C             int64
Heat Index - °C             int64
THW Index - °C              int64
Rain - mm                 float64
Rain Rate - mm/h          float64
ET - mm                   float64
Heating Degree Days       float64
Cooling Degree Days       float64
dtype: object

In [19]:
#use strings instead of typing :-) ---- needed for inserting into the database
formatStr = "(" + "%s,"*20 + "%s" +")"
formatStr

'(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'

Insert rows into the databse now. The format of the fields in the database is a wild guess from the data I had.  Cross check with the Vantage Pro documenation!

The code loads the data in the vantage table. If the vantage table already exists, it is removed before loading.

In [20]:
try:
    conn = msql.connect(user='root', password='omitted',
                              host='localhost', 
                              auth_plugin='mysql_native_password')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("USE CapWeather;")
        #destroy the old table
        cursor.execute('DROP TABLE IF EXISTS vantage;')
        print('Creating table....')
        #create a table in the database - wild guesses for format
        cursor.execute("CREATE TABLE vantage (date_time DATETIME NOT NULL, \
                        barometer FLOAT(8,2), \
                        temp_C TINYINT, \
                        high_temp_C TINYINT, \
                        low_temp_C TINYINT, \
                        hum_percent TINYINT, \
                        dew_point_C TINYINT, \
                        wet_bulb_C TINYINT, \
                        wind_speed_km_h SMALLINT, \
                        wind_direction VARCHAR(5), \
                        wind_run_km FLOAT(8,2), \
                        high_wind_speed_km_h  SMALLINT,  \
                        high_wind_direction VARCHAR(5), \
                        wind_chill_C TINYINT, \
                        heat_index_C TINYINT, \
                        THW_index_C TINYINT, \
                        rain_mm FLOAT(6,2), \
                        rain_rate_mm_h FLOAT(6,4), \
                        ET_mm  FLOAT(8,2), \
                        heating_degree_days FLOAT(8,4),  \
                        cooling_degree_days  FLOAT(8,4) \
                    )")
        print("vantage table is created....")
        
        #add the data from the csv file to the table - start with complete records
        for i,row in weatherDataComplete.iterrows():
            row[0] = datetime.datetime.strptime(row[0], '%m/%d/%y %I:%M %p')
           
            sql = "INSERT INTO CapWeather.vantage VALUES " + formatStr
            cursor.execute(sql, tuple(row))
            # the connection is not autocommitted by default, so we must commit to save our changes
            print(row[0])
            conn.commit()
            
        #add the rows with missing data (hopefully it is not date/time that is missing?)
        for i,row in weatherDataMissing.iterrows():
            row[0] = datetime.datetime.strptime(row[0], '%m/%d/%y %I:%M %p')
            
            #replace '--' by None - will be NULL in the db
            row.mask(row == '--',None, inplace = True)

            cursor.execute(sql, tuple(row))
    
            # the connection is not autocommitted by default, so we must commit to save our changes
            print(row[0])
            conn.commit()
            
  
except Error as e:
    print("Error while connecting to MySQL", e)

Creating table....
vantage table is created....
2020-01-01 00:00:00
2020-01-01 00:30:00
2020-01-01 01:00:00
2020-01-01 01:30:00
2020-01-01 02:00:00
2020-01-01 02:30:00
2020-01-01 03:00:00
2020-01-01 03:30:00
2020-01-01 04:00:00
2020-01-01 04:30:00
2020-01-01 05:00:00
2020-01-01 05:30:00
2020-01-01 06:00:00
2020-01-01 06:30:00
2020-01-01 07:00:00
2020-01-01 07:30:00
2020-01-01 08:00:00
2020-01-01 08:30:00
2020-01-01 09:00:00
2020-01-01 09:30:00
2020-01-01 10:00:00
2020-01-01 10:30:00
2020-01-01 11:00:00
2020-01-01 11:30:00
2020-01-01 12:00:00
2020-01-01 12:30:00
2020-01-01 13:00:00
2020-01-01 13:30:00
2020-01-01 14:00:00
2020-01-01 14:30:00
2020-01-01 15:00:00
2020-01-01 15:30:00
2020-01-01 16:00:00
2020-01-01 16:30:00
2020-01-01 17:00:00
2020-01-01 17:32:00
2020-01-01 18:00:00
2020-01-01 18:30:00
2020-01-01 19:00:00
2020-01-01 19:30:00
2020-01-01 21:00:00
2020-01-01 21:30:00
2020-01-01 22:00:00
2020-01-01 22:30:00
2020-01-01 23:00:00
2020-01-02 02:00:00
2020-01-02 02:30:00
2020-01-02 0

2020-01-14 01:30:00
2020-01-14 02:00:00
2020-01-14 02:30:00
2020-01-14 03:00:00
2020-01-14 03:30:00
2020-01-14 04:00:00
2020-01-14 04:30:00
2020-01-14 05:00:00
2020-01-14 05:30:00
2020-01-14 06:00:00
2020-01-14 06:30:00
2020-01-14 07:00:00
2020-01-14 07:30:00
2020-01-14 08:00:00
2020-01-14 08:30:00
2020-01-14 09:00:00
2020-01-14 09:30:00
2020-01-14 10:00:00
2020-10-08 13:00:00
2020-10-08 13:30:00
2020-10-08 15:00:00
2020-10-08 15:30:00
2020-10-08 16:00:00
2020-10-08 16:30:00
2020-10-08 17:00:00
2020-10-08 17:30:00
2020-10-08 18:00:00
2020-10-08 18:30:00
2020-10-08 19:00:00
2020-10-08 19:30:00
2020-10-08 20:00:00
2020-10-08 20:30:00
2020-10-08 21:00:00
2020-10-08 21:30:00
2020-10-08 22:00:00
2020-10-08 22:30:00
2020-10-08 23:00:00
2020-10-08 23:30:00
2020-10-09 00:00:00
2020-10-09 00:30:00
2020-10-09 01:00:00
2020-10-09 01:30:00
2020-10-09 02:00:00
2020-10-09 02:30:00
2020-10-09 03:00:00
2020-10-09 03:30:00
2020-10-09 04:00:00
2020-10-09 04:30:00
2020-10-09 05:00:00
2020-10-09 05:30:00


2020-10-16 23:00:00
2020-10-16 23:30:00
2020-10-17 00:00:00
2020-10-17 00:30:00
2020-10-17 01:00:00
2020-10-17 01:30:00
2020-10-17 02:00:00
2020-10-17 02:30:00
2020-10-17 03:00:00
2020-10-17 03:30:00
2020-10-17 04:00:00
2020-10-17 04:30:00
2020-10-17 05:00:00
2020-10-17 05:30:00
2020-10-17 06:00:00
2020-10-17 06:30:00
2020-10-17 07:00:00
2020-10-17 07:30:00
2020-10-17 08:00:00
2020-10-17 08:30:00
2020-10-17 09:00:00
2020-10-17 09:30:00
2020-10-17 10:00:00
2020-10-17 10:30:00
2020-10-17 11:00:00
2020-10-17 11:30:00
2020-10-17 12:00:00
2020-10-17 12:30:00
2020-10-17 13:00:00
2020-10-17 13:30:00
2020-10-17 14:00:00
2020-10-17 14:30:00
2020-10-17 15:00:00
2020-10-17 15:30:00
2020-10-17 16:00:00
2020-10-17 16:30:00
2020-10-17 17:00:00
2020-10-17 17:30:00
2020-10-17 18:00:00
2020-10-17 18:30:00
2020-10-17 19:00:00
2020-10-17 19:30:00
2020-10-17 20:00:00
2020-10-17 20:30:00
2020-10-17 21:00:00
2020-10-17 21:30:00
2020-10-17 22:00:00
2020-10-17 22:30:00
2020-10-17 23:00:00
2020-10-17 23:30:00


2020-10-26 09:30:00
2020-10-26 10:00:00
2020-10-26 10:30:00
2020-10-26 11:00:00
2020-10-26 11:30:00
2020-10-26 12:00:00
2020-10-26 12:30:00
2020-10-26 13:00:00
2020-10-26 13:30:00
2020-10-26 14:00:00
2020-10-26 14:30:00
2020-10-26 15:00:00
2020-10-26 15:30:00
2020-10-26 16:00:00
2020-10-26 16:30:00
2020-10-26 17:00:00
2020-10-26 17:30:00
2020-10-26 18:00:00
2020-10-26 18:30:00
2020-10-26 19:00:00
2020-10-26 19:30:00
2020-10-26 20:00:00
2020-10-26 20:30:00
2020-10-26 21:00:00
2020-10-26 21:30:00
2020-10-26 22:00:00
2020-10-26 22:30:00
2020-10-26 23:00:00
2020-10-26 23:30:00
2020-10-27 00:00:00
2020-10-27 00:30:00
2020-10-27 01:00:00
2020-10-27 01:30:00
2020-10-27 02:00:00
2020-10-27 02:30:00
2020-10-27 03:00:00
2020-10-27 03:30:00
2020-10-27 04:00:00
2020-10-27 04:30:00
2020-10-27 05:00:00
2020-10-27 05:30:00
2020-10-27 06:00:00
2020-10-27 06:30:00
2020-10-27 07:00:00
2020-10-27 07:30:00
2020-10-27 08:00:00
2020-10-27 08:30:00
2020-10-27 09:00:00
2020-10-27 09:30:00
2020-10-27 10:00:00


2020-11-04 20:00:00
2020-11-04 20:30:00
2020-11-04 21:00:00
2020-11-04 21:30:00
2020-11-04 22:00:00
2020-11-04 22:30:00
2020-11-04 23:00:00
2020-11-04 23:30:00
2020-11-05 00:00:00
2020-11-05 00:30:00
2020-11-05 01:00:00
2020-11-05 01:30:00
2020-11-05 02:00:00
2020-11-05 02:30:00
2020-11-05 03:00:00
2020-11-05 03:30:00
2020-11-05 04:00:00
2020-11-05 04:30:00
2020-11-05 05:00:00
2020-11-05 05:30:00
2020-11-05 06:00:00
2020-11-05 06:30:00
2020-11-05 07:00:00
2020-11-05 07:30:00
2020-11-05 08:00:00
2020-11-05 08:30:00
2020-11-05 09:00:00
2020-11-05 09:30:00
2020-11-05 10:00:00
2020-11-05 10:30:00
2020-11-05 11:00:00
2020-11-05 11:30:00
2020-11-05 12:00:00
2020-11-05 12:30:00
2020-11-05 13:00:00
2020-11-05 13:30:00
2020-11-05 14:00:00
2020-11-05 14:30:00
2020-11-05 15:00:00
2020-11-05 15:30:00
2020-11-05 16:00:00
2020-11-05 16:30:00
2020-11-05 17:00:00
2020-11-05 17:30:00
2020-11-05 18:00:00
2020-11-05 18:30:00
2020-11-05 19:00:00
2020-11-05 19:30:00
2020-11-05 20:00:00
2020-11-05 20:30:00


2020-11-17 03:30:00
2020-11-17 04:00:00
2020-11-17 04:30:00
2020-11-17 05:00:00
2020-11-17 05:30:00
2020-11-17 06:00:00
2020-11-17 06:30:00
2020-11-17 07:00:00
2020-11-17 07:30:00
2020-11-17 08:00:00
2020-11-17 08:30:00
2020-11-17 09:00:00
2020-11-17 09:30:00
2020-11-17 10:00:00
2020-11-17 10:30:00
2020-11-17 11:00:00
2020-11-17 11:30:00
2020-11-17 12:00:00
2020-11-17 12:30:00
2020-11-17 13:00:00
2020-11-17 13:30:00
2020-11-17 14:00:00
2020-11-17 14:30:00
2020-11-17 15:00:00
2020-11-17 15:30:00
2020-11-17 16:00:00
2020-11-17 16:30:00
2020-11-17 17:00:00
2020-11-17 17:30:00
2020-11-17 18:00:00
2020-11-17 18:30:00
2020-11-17 19:00:00
2020-11-17 19:30:00
2020-11-17 20:00:00
2020-11-17 20:30:00
2020-11-17 21:00:00
2020-11-17 21:30:00
2020-11-17 22:00:00
2020-11-17 22:30:00
2020-11-17 23:00:00
2020-11-17 23:30:00
2020-11-18 00:00:00
2020-11-18 00:30:00
2020-11-18 01:00:00
2020-11-18 01:30:00
2020-11-18 02:00:00
2020-11-18 02:30:00
2020-11-18 03:00:00
2020-11-18 03:30:00
2020-11-18 04:00:00


2020-11-27 01:30:00
2020-11-27 02:00:00
2020-11-27 02:30:00
2020-11-27 03:00:00
2020-11-27 03:30:00
2020-11-27 04:00:00
2020-11-27 04:30:00
2020-11-27 05:00:00
2020-11-27 05:30:00
2020-11-27 06:00:00
2020-11-27 06:30:00
2020-11-27 07:00:00
2020-11-27 07:30:00
2020-11-27 08:00:00
2020-11-27 08:30:00
2020-11-27 09:00:00
2020-11-27 09:30:00
2020-11-27 10:00:00
2020-11-27 10:30:00
2020-11-27 11:00:00
2020-11-27 11:30:00
2020-11-27 12:00:00
2020-11-27 12:30:00
2020-11-27 13:00:00
2020-11-27 13:30:00
2020-11-27 14:00:00
2020-11-27 14:30:00
2020-11-27 15:00:00
2020-11-27 15:30:00
2020-11-27 16:00:00
2020-11-27 16:30:00
2020-11-27 17:00:00
2020-11-27 17:30:00
2020-11-27 18:00:00
2020-11-27 18:30:00
2020-11-27 19:00:00
2020-11-27 19:30:00
2020-11-27 20:00:00
2020-11-27 20:30:00
2020-11-27 21:00:00
2020-11-27 21:30:00
2020-11-27 22:00:00
2020-11-27 22:30:00
2020-11-27 23:00:00
2020-11-27 23:30:00
2020-11-28 00:00:00
2020-11-28 00:30:00
2020-11-28 01:00:00
2020-11-28 01:30:00
2020-11-28 02:00:00


2020-12-06 16:30:00
2020-12-06 17:00:00
2020-12-06 17:30:00
2020-12-06 18:00:00
2020-12-06 18:30:00
2020-12-06 19:00:00
2020-12-06 19:30:00
2020-12-06 20:00:00
2020-12-06 20:30:00
2020-12-06 21:00:00
2020-12-06 21:30:00
2020-12-06 22:00:00
2020-12-06 22:30:00
2020-12-06 23:00:00
2020-12-06 23:30:00
2020-12-07 00:00:00
2020-12-07 00:30:00
2020-12-07 01:00:00
2020-12-07 01:30:00
2020-12-07 02:00:00
2020-12-07 02:30:00
2020-12-07 03:00:00
2020-12-07 03:30:00
2020-12-07 04:00:00
2020-12-07 04:30:00
2020-12-07 05:00:00
2020-12-07 05:30:00
2020-12-07 06:00:00
2020-12-07 06:30:00
2020-12-07 07:00:00
2020-12-07 07:30:00
2020-12-07 08:00:00
2020-12-07 08:30:00
2020-12-07 09:00:00
2020-12-07 09:30:00
2020-12-07 10:00:00
2020-12-07 10:30:00
2020-12-07 11:00:00
2020-12-07 11:30:00
2020-12-07 12:00:00
2020-12-07 12:30:00
2020-12-07 13:00:00
2020-12-07 13:30:00
2020-12-07 14:00:00
2020-12-07 14:30:00
2020-12-07 15:00:00
2020-12-07 15:30:00
2020-12-07 16:00:00
2020-12-07 16:30:00
2020-12-07 17:00:00


2020-12-16 21:00:00
2020-12-16 21:30:00
2020-12-16 22:00:00
2020-12-16 22:30:00
2020-12-16 23:00:00
2020-12-16 23:30:00
2020-12-17 00:00:00
2020-12-17 00:30:00
2020-12-17 01:00:00
2020-12-17 01:30:00
2020-12-17 02:00:00
2020-12-17 02:30:00
2020-12-17 03:00:00
2020-12-17 03:30:00
2020-12-17 04:00:00
2020-12-17 04:30:00
2020-12-17 05:00:00
2020-12-17 05:30:00
2020-12-17 06:00:00
2020-12-17 06:30:00
2020-12-17 07:00:00
2020-12-17 07:30:00
2020-12-17 08:00:00
2020-12-17 08:30:00
2020-12-17 09:00:00
2020-12-17 09:30:00
2020-12-17 10:00:00
2020-12-17 10:30:00
2020-12-17 11:00:00
2020-12-17 11:30:00
2020-12-17 12:00:00
2020-12-17 12:30:00
2020-12-17 13:00:00
2020-12-17 13:30:00
2020-12-17 14:00:00
2020-12-17 14:30:00
2020-12-17 15:00:00
2020-12-17 15:30:00
2020-12-17 16:00:00
2020-12-17 16:30:00
2020-12-17 17:00:00
2020-12-17 17:30:00
2020-12-17 18:00:00
2020-12-17 18:30:00
2020-12-17 19:00:00
2020-12-17 19:30:00
2020-12-17 20:00:00
2020-12-17 20:30:00
2020-12-17 21:00:00
2020-12-17 21:30:00


2020-12-25 15:30:00
2020-12-25 16:00:00
2020-12-25 16:30:00
2020-12-25 17:00:00
2020-12-25 17:30:00
2020-12-25 18:00:00
2020-12-25 18:30:00
2020-12-25 19:00:00
2020-12-25 19:30:00
2020-12-25 20:00:00
2020-12-25 20:30:00
2020-12-25 21:00:00
2020-12-25 21:30:00
2020-12-25 22:00:00
2020-12-25 22:30:00
2020-12-25 23:00:00
2020-12-25 23:30:00
2020-12-26 00:00:00
2020-12-26 00:30:00
2020-12-26 01:00:00
2020-12-26 01:30:00
2020-12-26 02:00:00
2020-12-26 02:30:00
2020-12-26 03:00:00
2020-12-26 03:30:00
2020-12-26 04:00:00
2020-12-26 04:30:00
2020-12-26 05:00:00
2020-12-26 05:30:00
2020-12-26 06:00:00
2020-12-26 06:30:00
2020-12-26 07:00:00
2020-12-26 07:30:00
2020-12-26 08:00:00
2020-12-26 08:30:00
2020-12-26 09:00:00
2020-12-26 09:30:00
2020-12-26 10:00:00
2020-12-26 10:30:00
2020-12-26 11:00:00
2020-12-26 11:30:00
2020-12-26 12:00:00
2020-12-26 12:30:00
2020-12-26 13:00:00
2020-12-26 13:30:00
2020-12-26 14:00:00
2020-12-26 14:30:00
2020-12-26 15:00:00
2020-12-26 15:30:00
2020-12-26 16:00:00
